In [13]:
import requests
import xml.etree.ElementTree as ET
from html.parser import HTMLParser

In [36]:

def download_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.capture_data = False
        self.target_class = ""
        self.captured_data = []

    def handle_starttag(self, tag, attrs):
        if tag == "span":
            for attr in attrs:
                if attr[0] == "class" and attr[1] == self.target_class:
                    self.capture_data = True

    def handle_endtag(self, tag):
        if tag == "span" and self.capture_data:
            self.capture_data = False

    def handle_data(self, data):
        if self.capture_data:
            self.captured_data.append(data)

    def extract_span_by_class(self, html, class_value):
        self.target_class = class_value
        self.captured_data = []
        self.feed(html)
        return self.captured_data


In [31]:

url = "https://www.example.com"
url = "https://filesender.aarnet.edu.au/?s=download&token=d6be9395-9fc6-427f-b565-289cb32a16e2"
html_content = download_html(url)

In [44]:

parser = MyHTMLParser()
directlinks = parser.extract_span_by_class(html_content, "directlink")
directlinks = [x.split("Direct Link: ")[1].strip() for x in directlinks]
token = url.split("&token=")[1]
fileids = [x.split("&files_ids=")[1] for x in directlinks]
